In [1]:
import pandas as pd
import numpy as np
import os
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
combined_energy_consumption = pd.read_csv('/content/drive/MyDrive/Data Science/combined_energy_consumption.csv')
demand_energy_consumption = pd.read_csv('/content/drive/MyDrive/Data Science/demand_energy_consumption.csv')


In [3]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [5]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Energy Consumption Dashboard", style = {'textAlign': 'center'}),
    html.Label("Choose Aggregation Interval:", style = {'fontSize': '18px'}),
    dcc.Dropdown(
        id = 'time-dropdown',
        options = [
            {'label': 'Hourly', 'value': 'hour'},
            {'label': 'By Day Type', 'value': 'daytype'},
            {'label': 'Monthly', 'value': 'month'},
            {'label': 'Seasonal', 'value': 'season'}
        ],
        value = 'hour',
        clearable = False
    ),
    dcc.Graph(id = 'energy-graph')
])

def aggregate_data(data, interval):
    if interval == 'hour':
        aggregated = data.groupby('Hour')['KWH/hh_corrected'].mean().reset_index()
    elif interval == 'daytype':
        aggregated = data.groupby('DayType')['KWH/hh_corrected'].mean().reset_index()
    elif interval == 'month':
        aggregated = data.groupby('Month')['KWH/hh_corrected'].mean().reset_index()
    elif interval == 'season':
        aggregated = data.groupby('Season')['KWH/hh_corrected'].mean().reset_index()
    else:
        raise ValueError("Unsupported interval")
    return aggregated

@app.callback(
    Output('energy-graph', 'figure'),
    Input('time-dropdown', 'value')
)
def update_graph(selected_interval):
    aggregated_data = aggregate_data(combined_energy_consumption, selected_interval)

    if selected_interval == 'hour':
        fig = px.line(aggregated_data, x = 'Hour', y = 'KWH/hh_corrected',
                      title = 'Average Energy Consumption by Hour',
                      labels = {'KWH/hh_corrected': 'Average Energy Consumption (kWh/hh)', 'Hour': 'Hour of Day'})
        fig.update_xaxes(dtick = 1)
    elif selected_interval == 'daytype':
        fig = px.bar(aggregated_data, x = 'DayType', y = 'KWH/hh_corrected',
                     title = 'Average Energy Consumption by Day Type',
                     labels = {'KWH/hh_corrected': 'Average Energy Consumption (kWh/hh)', 'DayType': 'Day Type'})
    elif selected_interval == 'month':
        fig = px.bar(aggregated_data, x = 'Month', y = 'KWH/hh_corrected',
                     title = 'Average Energy Consumption by Month',
                     labels = {'KWH/hh_corrected': 'Average Energy Consumption (kWh/hh)', 'Month': 'Month'})
    elif selected_interval == 'season':
        fig = px.bar(aggregated_data, x ='Season', y = 'KWH/hh_corrected',
                     title = 'Average Energy Consumption by Season',
                     labels = {'KWH/hh_corrected': 'Average Energy Consumption (kWh/hh)', 'Season': 'Season'})
    return fig

if __name__ == '__main__':
    app.run_server(debug = True, port = 8050)


<IPython.core.display.Javascript object>

In [8]:
demand_cost = demand_energy_consumption.groupby('Demand')['TotalCost'].sum().reset_index()
demand_hour_cost = demand_energy_consumption.groupby(['Demand', 'Hour'])['TotalCost'].mean().reset_index()



In [9]:
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Energy and Cost Savings Dashboard"),

    html.Div([
        html.Label("Select Threshold Percentile:"),
        dcc.Slider(
            id = 'threshold-slider',
            min = 0,
            max = 1,
            step = 0.05,
            value = 0.25,
            marks = {i / 10: f"{i * 10}%" for i in range(0, 11)}
        )
    ]),

    html.Div(id = 'savings-summary'),

    dcc.Graph(id = 'pie-chart'),

    dcc.Graph(id = 'line-plot')
])

@app.callback(
    [
        Output('savings-summary', 'children'),
        Output('pie-chart', 'figure'),
        Output('line-plot', 'figure')
    ],
    [Input('threshold-slider', 'value')]
)
def update_dashboard(threshold):
    non_zero_data = demand_energy_consumption[demand_energy_consumption['KWH/hh_corrected'] > 0]
    dynamic_threshold = non_zero_data['KWH/hh_corrected'].quantile(threshold)

    low_energy_rows = demand_energy_consumption[demand_energy_consumption['KWH/hh_corrected'] <= dynamic_threshold]

    price_mapping = {'High': 0.7721, 'Normal': 0.1946, 'Low': 0.0689}
    low_energy_rows['DemandPrice'] = low_energy_rows['Demand'].map(price_mapping)
    low_energy_rows['EnergySaved'] = low_energy_rows['KWH/hh_corrected']
    low_energy_rows['CostSaved'] = low_energy_rows['EnergySaved'] * low_energy_rows['DemandPrice']

    total_energy_saved = low_energy_rows['EnergySaved'].sum()
    total_cost_saved = low_energy_rows['CostSaved'].sum()

    summary = html.Div([
        html.H3(f"Dynamic Threshold: {dynamic_threshold:.2f} kWh"),
        html.P(f"Total Energy Saved: {total_energy_saved:.2f} kWh"),
        html.P(f"Total Cost Saved: {total_cost_saved:.2f} EGP")
    ])

    pie_chart = px.pie(
        demand_cost,
        values = 'TotalCost',
        names = 'Demand',
        title = 'Demand Percentage and Original Total Cost',
        hole = 0.4
    )

    line_plot = px.line(
        demand_hour_cost,
        x = 'Hour',
        y = 'TotalCost',
        color = 'Demand',
        title = 'Average Energy Cost by Demand and Hour of Day',
        labels = {
            'Hour': 'Hour of Day',
            'TotalCost': 'Average Cost (EGP)'
        }
    )

    return summary, pie_chart, line_plot

if __name__ == '__main__':
    app.run_server(debug = True, port = 8051)


<IPython.core.display.Javascript object>